In [28]:
##Estimación de indicadores de vivienda
#Se borra todo lo que se encuentra en el entorno

rm(list=ls())

# Librerías ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, showtext, srvyr, haven)

In [8]:
#Crear directorio para guardar los archivos
dir.create("data", showWarnings = FALSE)

In [9]:
url_basica<-"https://www.inegi.org.mx/contenidos/programas/enigh/nc/"
segunda_parte<-"microdatos/enigh"
fin1<-"_ns_"
fin2<-"_csv.zip"

In [10]:
years<-c("2016", "2018", "2020", "2022")

In [11]:
# Función genérica para descargar, descomprimir, renombrar y eliminar
procesar_archivo <- function(year, tipo, url_base, segunda_parte, fin1, fin2, carpeta_destino) {
  # Construir URL y ruta de destino
  url <- paste0(url_base, year, "/", segunda_parte, year, fin1, tipo, fin2)
  destfile <- paste0(carpeta_destino, "/", year, fin1, tipo, fin2)
  
  # Descargar el archivo
  download.file(url, destfile = destfile)
  
  # Descomprimir el archivo
  unzip(destfile, exdir = carpeta_destino)
  
  # Renombrar el archivo descomprimido
  file.rename(
    from = paste0(carpeta_destino, "/", tipo, ".csv"),
    to = paste0(carpeta_destino, "/", tipo, "_", year, ".csv")
  )
  
  # Eliminar el archivo descargado
  file.remove(destfile)
}

In [12]:
# Iterar sobre los años y procesar cada tipo de archivo
for (i in 1:length(years)) {
  procesar_archivo(years[i], "poblacion", url_basica, segunda_parte, fin1, fin2, "data")
  procesar_archivo(years[i], "viviendas", url_basica, segunda_parte, fin1, fin2, "data")
  procesar_archivo(years[i], "trabajos", url_basica, segunda_parte, fin1, fin2, "data")
}

### Función de estimación

In [40]:
estimacion <- function(path, variable) {
  # Cargar los datos de población
  datos <- read_csv(path, show_col_types = FALSE)
  
  # Si son datos de 2016 a 2020, unir con la tabla de viviendas
  if (substr(path, 16, 19) %in% c("2016", "2018", "2020")) {
    # Cargar tabla viviendas
    vivienda <- read_csv(str_replace(path, "poblacion", "viviendas"), show_col_types = FALSE)
    
    # Pegar factor, upm y est_dis de la tabla vivienda
    datos <- datos %>%
      left_join(vivienda %>% select(folioviv, factor, upm, est_dis), by = "folioviv")
  }
  
  # Unir con la tabla trabajos si es "ocupacion" o si la persona debe ser identificada como indígena
  if (variable == "ocupacion") {
    # Cargar tabla trabajos
    trabajos <- read_csv(str_replace(path, "poblacion", "trabajos"), show_col_types = FALSE)
    
    # Pegar datos relevantes de la tabla trabajos
    datos <- datos %>%
      left_join(trabajos %>% select(folioviv, foliohog, numren, id_trabajo, trapais), 
                by = c("folioviv", "foliohog", "numren"))
  }
  
  # Crear variable cve_ent
  datos <- datos %>%
    mutate(cve_ent = case_when(
      nchar(folioviv) == 9  ~ paste0("0", substr(folioviv, 1, 1)),
      nchar(folioviv) == 10 ~ substr(folioviv, 1, 2)
    )) %>%
    # Filtrar si entidad es CDMX (09)
    filter(cve_ent == "09")
  
  # Definir diseño muestral
  mydesign <- datos %>%
    as_survey_design(ids = upm,
                     strata = est_dis,
                     weights = factor)
  
  # Estimar población hablante indígena en CDMX
  if (variable == "hablante") {
    hablante <- mydesign %>%
      group_by(hablaind) %>%
      summarise(
        poblacion = survey_total(vartype = "cv"),
        pct = survey_prop(vartype = "cv")
      ) %>%
      mutate(pct = round(pct * 100, 2)) %>%
      # Añadir año
      mutate(year = substr(path, 16, 19)) %>%
      filter(hablaind == 1)
    
    return(hablante)
  }
  
    # Estimar población hablante indígena en CDMX
  if (variable == "hablante_indigena") {
    hablante <- mydesign %>%
      group_by(etnia,hablaind) %>%
      summarise(
        poblacion = survey_total(vartype = "cv"),
        pct = survey_prop(vartype = "cv")
      ) %>%
      mutate(pct = round(pct * 100, 2)) %>%
      # Añadir año
      mutate(year = substr(path, 16, 19)) %>%
      filter(etnia == 1)%>%
filter(hablaind == 1)
    
    return(hablante)
  }
  
  
  # Estimar población ocupada, considerando si la persona es indígena
  if (variable == "ocupacion") {
    ocupacion <- mydesign %>%
      group_by(trapais,etnia) %>%
      summarise(
        ocupados = survey_total(vartype = "cv"),
        pct = survey_prop(vartype = "cv")
      ) %>%
      mutate(pct = round(pct * 100, 2)) %>%
      # Añadir año
      mutate(year = substr(path, 16, 19)) %>%
      filter(trapais == 1)%>%
filter(etnia == 1) 
    return(ocupacion)
  }
  
  # Estimar población analfabeta
  if (variable == "analfabetismo") {
    analfa <- mydesign %>%
      group_by(etnia,alfabetism) %>%
      summarise(
        analfabetas = survey_total(vartype = "cv"),
        pct = survey_prop(vartype = "cv")
      ) %>%
      mutate(pct = round(pct * 100, 2)) %>%
      # Añadir año
      mutate(year = substr(path, 16, 19)) %>%
filter(etnia == 1)%>% 
filter(alfabetism == 2)
    return(analfa)
  }  
  
}


#### Población hablante indígena en CDMX

In [14]:
tabla_hablante<-map_dfr(
  list.files("data", full.names = TRUE, pattern = "poblacion_.*.csv"),
  estimacion,
  variable = "hablante"
) 
print("Población hablante de lengua indígena en la CDMX")
tabla_hablante

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"


[1] "Población hablante de lengua indígena en la CDMX"


hablaind,poblacion,poblacion_cv,pct,pct_cv,year
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,129097,0.1508605,1.41,0.1505873,2016
1,127656,0.2158292,1.38,0.2139413,2018
1,154656,0.1536397,1.67,0.1530732,2020
1,134098,0.1518493,1.43,0.1521503,2022


#### Población hablante y autodescripción indígena en CDMX

In [41]:
tabla_hablante_indigena<-map_dfr(
  list.files("data", full.names = TRUE, pattern = "poblacion_.*.csv"),
  estimacion,
  variable = "hablante_indigena"
)
print("Población según autodescripción y condición de hablante de lengua indígena en la CDMX")
tabla_hablante_indigena

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"There was 1 warning in `dplyr::summarise()`.
ℹ In argument: `pct = survey_prop(vartype = "cv")`.
ℹ In group 5: `etnia = NA` and `hablaind = NA`.
Caused by warning:
! glm.fit: algorithm did not converge"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"There was 1 warning in `dplyr::summarise()`.
ℹ In argument: `pct = survey_prop(vartype = "cv")`.
ℹ In group 5: `etnia = NA` and `hablaind = NA`.
Caused by warning:
! glm.fit: algorithm did not converge"
Wa

[1] "Población según autodescripción y condición de hablante de lengua indígena en la CDMX"


etnia,hablaind,poblacion,poblacion_cv,pct,pct_cv,year
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,1,110147,0.1701360,8.81,0.1710783,2016
1,1,107151,0.2531187,7.83,0.2424770,2018
1,1,139719,0.1634319,10.03,0.1551507,2020
1,1,116757,0.1632034,10.58,0.1590305,2022


#### Población ocupada indígena en CDMX

In [16]:
tabla_ocupacion<-map_dfr(
  list.files("data", full.names = TRUE, pattern = "poblacion_.*.csv"),
  estimacion,
  variable = "ocupacion"
)
print("Población que se autodescribe como indígena y que se encuentra ocupada en la CDMX")
tabla_ocupacion

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or mo

[1] "Población que se autodescribe como indígena y que se encuentra ocupada en la CDMX"


trapais,etnia,ocupados,ocupados_cv,pct,pct_cv,year
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,1,701096,0.08687768,15.24,0.08163274,2016
1,1,716466,0.08822712,15.83,0.08266348,2018
1,1,684724,0.06773705,16.27,0.06419507,2020
1,1,550736,0.08300888,11.86,0.08205935,2022


####   Población analfabeta indígena en CDMX

In [26]:
tabla_analfabetismo<-map_dfr(
  list.files("data", full.names = TRUE, pattern = "poblacion_.*.csv"),
  estimacion,
  variable = "analfabetismo"
)
print("Población que se autodescribe como indígena y que es analfabeta en la CDMX")

tabla_analfabetismo

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"There was 1 warning in `dplyr::summarise()`.
ℹ In argument: `pct = survey_prop(vartype = "cv")`.
ℹ In group 5: `etnia = NA` and `alfabetism = NA`.
Caused by warning:
! glm.fit: algorithm did not converge"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"There was 1 warning in `dplyr::summarise()`.
ℹ In argument: `pct = survey_prop(vartype = "cv")`.
ℹ In group 5: `etnia = NA` and `alfabetism = NA`.
Caused by warning:
! glm.fit: algorithm did not converge

[1] "Población que se autodescribe como indígena y que es analfabeta en la CDMX"


etnia,alfabetism,analfabetas,analfabetas_cv,pct,pct_cv,year
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,2,76848,0.2047636,6.14,0.1804278,2016
1,2,100372,0.1583444,7.33,0.1251706,2018
1,2,91929,0.1348095,6.60,0.1153417,2020
1,2,63920,0.2065419,5.79,0.1828260,2022
